# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1

In [1]:
#Libraries 
from bs4 import BeautifulSoup #Library for pulling data out of HTML and XML file
import requests # library to handle requests
import pandas as pd #library for data analsysis
import numpy as np #library to handle data in a vectorized manner
import folium #Map rendering
from geopy.geocoders import Nominatim #Library to geolocate a query
from pandas.io.json import json_normalize #Library for normalizing semi-structured JSON data into a flat table.
from sklearn.cluster import KMeans #Library kmeans ...
from matplotlib import cm,colors

### Build the code to scrape the postal codes of Canada

In [2]:
#List of postal codes of Canada: M
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Get html from wiki page and create soup object
source=requests.get(url).text
soup = BeautifulSoup(source, 'xml')

#Define the dataframe columns
column_names=['PostalCode','Borough','Neighborhood']

#Instantiate the dataframe
df=pd.DataFrame(columns = column_names)

#Collect postal code, borough, neighborhood data 
for tr in soup.find_all('tr'):
    row_data=[]
    for td_cell in tr.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        
#Show the first five rows of the dataframe
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Ignore cells with a borough that is "not assigned"
df = df[df.Borough != 'Not assigned']

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df['Neighborhood'].replace("Not assigned",df["Borough"],inplace=True)

#Reindex and show the first five rows of the dataset
df.reset_index(drop=True).head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
#Show the number of rows and columns of the dataframe
print('Number of rows and columns of the dataframe :',df.shape)

Number of rows and columns of the dataframe : (103, 3)


# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

## Part 2

### Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, get the latitude and the longitude coordinates of each neighborhood. 

In [5]:
#Read geospatial coordinates csv file
df_coord = pd.read_csv('/Users/juliagau/Desktop/Projets/Coursera_Capstone/Geospatial_Coordinates.csv')

#Rename Postal Code column into PostalCode
df_coord.rename(columns={'Postal Code':'PostalCode'},inplace=True)

#Merge df and df_coord dataframe 
df_coord_merged = pd.merge(df, df_coord,on='PostalCode')

#Show the first five rows of the dataframe with postal code, borough, neighborhood, latitude and logitude data
df_coord_merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

## Part 3

### Exploration and clustering the neighborhoods in Toronto. I decide to work only with the boroughs that contain the word Toronto.

In [6]:
#Filter data to only use boroughs in Toronto and drop PostalCode column
df_toronto = df_coord_merged[df_coord_merged['Borough'].str.contains('Toronto')].drop(['PostalCode'], axis = 1).reset_index(drop=True)

#Show the first five rows of the dataframe
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [7]:
#Create an intial map of Toronto using first entries latitude and longitude values of the neighborhoods in Toronto
map_toronto = folium.Map(location=[df_toronto["Latitude"][0], df_toronto["Longitude"][0]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
#Start utilizing the Foursquare API to explore the neighborhoods and segment them

CLIENT_ID='ZVUJ2SL1JDLBYKV51ZQZA4SEM4ELAIMS3T5CQO14D00EIL3M' #Foursquare ID
CLIENT_SECRET='ASSXNQPBNYPKQF0UWGIAQR2VZ2H1EJGGEPX0EVO111LWZWPJ' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


My credentails:
CLIENT_ID: ZVUJ2SL1JDLBYKV51ZQZA4SEM4ELAIMS3T5CQO14D00EIL3M
CLIENT_SECRET:ASSXNQPBNYPKQF0UWGIAQR2VZ2H1EJGGEPX0EVO111LWZWPJ


In [9]:
# Get data of first neighborhood
ng_latitude=df_toronto['Latitude'][0] #Neighborhood latitude value
ng_longitude=df_toronto['Longitude'][0] #neighborhood longitude value
ng_name = df_toronto['Neighborhood'][0] #Neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(ng_name, 
                                                               ng_latitude, 
                                                               ng_longitude))



Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [10]:
#Function for the getting values

LIMIT = 100 #Limit of number of venues returned by Foursquare API
radius = 500 #Define radius

#Create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ng_latitude, 
    ng_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
#results

#Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
        
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

venues=results['response']['groups'][-1]['items']
    
nearby_venues=json_normalize(venues) #Flatten JSON

#Filter columns
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]

#Filter the category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)

#Clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

#Show the first five rows of the dataframe
nearby_venues.head()

#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))




,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [12]:
#To all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues=getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude']
                                  )

#Size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

#Unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [13]:
#Analyze each neighborhood

#One hot encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#Move neighborhood column to the first column
fixed_columns=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]

#Show the first five rows of the dataframe
toronto_onehot.head()


,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.015873,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.015873,0.0


In [15]:
#Show each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3      Farmers Market  0.03
4              Bakery  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Skate Park  0.06
4          Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Sho

## Put all the data into a dataframe

In [16]:
#Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues=10
indicators=['st', 'nd', 'rd']

#Create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Create a new dataframe
toronto_venues_sorted=pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood']=toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:]=return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Beer Bar,Restaurant,Seafood Restaurant,Basketball Stadium,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Furniture / Home Store,Bar,Nightclub,Bakery,Gym,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Gym / Fitness Center,Skate Park,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Sculpture Garden,Plane,Coffee Shop,Bar
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant


In [17]:
#Cluster Neighborhoods

#Set number of clusters
kclusters=int(len(df_toronto["Neighborhood"].unique()) / 4)
toronto_grouped_clustering=toronto_grouped.drop('Neighborhood', 1)

#Run k-means clustering
kmeans=KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

#Add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged=df_toronto.copy()
toronto_merged.rename(columns={'Neighborhood':'Neighborhood'}, inplace=True)
#Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged=toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() #Check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Event Space,Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Fast Food Restaurant,Italian Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gym,Park,Italian Restaurant,Restaurant,Cosmetics Shop,Clothing Store
4,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Wine Shop,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop


In [18]:
#Visualization of the resulting clusters

#Create map
map_clusters=folium.Map(location=[df_toronto["Latitude"][0], df_toronto["Longitude"][0]], zoom_start=12)

#Set color scheme for the clusters
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

#Add markers to the map
markers_colors=[]
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label=folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [19]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Event Space,Restaurant
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Fast Food Restaurant,Italian Restaurant
3,St. James Town,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gym,Park,Italian Restaurant,Restaurant,Cosmetics Shop,Clothing Store
5,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Beer Bar,Restaurant,Seafood Restaurant,Basketball Stadium,Beach
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
8,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Gym,Thai Restaurant,Bakery,Steakhouse,Concert Hall
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Fried Chicken Joint,Restaurant,Scenic Lookout,Bar
12,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Japanese Restaurant,Caribbean Restaurant,Indian Restaurant,Spa
13,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,American Restaurant,Deli / Bodega
